In [ ]:
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
%load_ext autoreload


import pandas as pd  
import numpy as np  
from time import time
from sklearn.model_selection import train_test_split 
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
from torchvision import transforms, utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns

Sorting data 

In [ ]:
dataroot = '/home/jovyan/ikt590'
SEED = 2
EPOCH = 50           # go through the data n times
BATCH_SIZE = 4096     # batch
TIME_STEP = None      
INPUT_SIZE = 76         # number of features
LR = 0.01              # learning rate

Batch size må økes til -> 4096*4*5 
num_layers -> må økes test deg fram 
hidden_size -> må økes test deg fram


In [ ]:
from preprocessing import load_data
X,y = load_data(dataroot)

In [ ]:
from sklearn import preprocessing
from preprocessing import balance_data, normalize

X = normalize(X)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    def __len__(self):
        return(len(self.labels))
    def __getitem__(self, index):
        return self.features[index],self.labels[index]


In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.LSTM(
          input_size=INPUT_SIZE,
          hidden_size=32,         # hidden units
          num_layers=4,           # lstm layers
          dropout=0.3,
          batch_first=True,     
          )
        self.out = nn.Linear(32, 15)
    def forward(self, x):
        r_out, (h_n, h_c) = self.rnn(x, None)  
        out = self.out(r_out[:, -1, :])
        return out

rnn = RNN()
print(rnn)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics

K=5
skf = StratifiedKFold(n_splits=K, random_state=SEED, shuffle=True)
for fold_index, (train_index,test_index) in enumerate(skf.split(X,y)):
        print('---------------------------------------------')
        print('Fold #{}'.format(fold_index))    
        
        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]
        
        X_train = X_train.astype(float)
        y_train = y_train.astype(int)
        p = np.random.permutation(len(y_train))
        X_train = X_train[p]
        y_train = y_train[p]
        X_train,y_train = balance_data(X_train,y_train,seed=SEED)

        train_data = CustomDataset(X_train, y_train)
        test_data = CustomDataset(X_test, y_test)
        
        '''
        Bare ta vekk num_workers, sjekk denne linken om vi kan bruke noen til å splite datasettet i 5.
        https://stackoverflow.com/questions/58996242/cross-validation-for-mnist-dataset-with-pytorch-and-sklearn
        https://pytorch.org/docs/stable/data.html
        sjekk disse linkene burde fungere å gjøre det slik GLHF
        '''
        trainloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)
        testloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

        optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize rnn parameters
        loss_func = nn.CrossEntropyLoss()                       

        for epoch in range(EPOCH):

            for step, (b_x, b_y) in enumerate(trainloader):   

                b_x = b_x.view(-1, 1, INPUT_SIZE).float()       

                output = rnn(b_x)

                loss = loss_func(output,b_y)                   

                optimizer.zero_grad()

                loss.backward()                                 
                optimizer.step()                                

            ## Add splitfold here 
            y_predict=[]
            for step,(a_x, a_y) in enumerate(testloader):
                a_x = a_x.view(-1,1,INPUT_SIZE).float()
                test_output = rnn(a_x)
                pred_y = torch.max(test_output, 1)[1]
                y_predict=y_predict+pred_y.numpy().tolist()


        precision, recall, fscore, support = score(y_test, y_predict)
        precision=precision*100
        recall=recall*100
        print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy())
        for i in range(15):
            print('Label: ', i,'| precision: %.2f' % precision[i],'%','| recall: %.2f' % recall[i],'%','| F1: %.2f' % fscore[i],'| support:', support[i])

        # Cross-validation 
        test_acc = metrics.balanced_accuracy_score(y_test,y_predict)*100
        print('balanced test set accuracy: ',test_acc)
        
        # Lables
        target_names = ['Benign', 'Botnet', 'DDoS', 'DoS GoldenEye', 'DoS Hulk',
                            'DoS Slowhttptest', 'DoS slowloris', 'FTP-Patator', 'Heartbleed', 
                            'Infiltration', 'PortScan', 'SSH-Patator', 'Web Attack - Brute Force',
                            'Web Attack - Sql Injection', 'Web Attack - XSS']
        # Confusion metrics
        print(classification_report(y_test, y_predict, target_names=target_names))
        # Confusion metrics heat map          
        conf_matrix = confusion_matrix(y_test, y_predict)
        cm_df = pd.DataFrame(conf_matrix)
        plt.figure(figsize=(20,15))
        sns.set(font_scale=1.4)
        sns.heatmap(cm_df, annot=True, annot_kws={"size":12}, fmt='g', xticklabels=target_names,
                                yticklabels=target_names)
        plt.ylabel('Actual Class')
        plt.xlabel('Predicted Class')
        plt.show()